<a href="https://colab.research.google.com/github/aayushs879/Soc-Gen-Complaint-Status-Tracking/blob/master/socgen_language_wise_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#mounting colab book at google drive
from google.colab import drive
drive.mount('/content/gdrive')
import os #better data flow
path = os.path.abspath('gdrive/My Drive/HackerEarth/Soc-Gen')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
#importing the libraries and downloading stopwords

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import f1_score as f1s

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#importing the dataset and test set which is to be submitted
train = pd.read_csv(os.path.join(path, 'train.csv'))
test = pd.read_csv(os.path.join(path, 'test.csv'))

#extracting and eliminating the target variable
y = train['Complaint-Status'].iloc[:].values
train = train.drop(['Complaint-Status'], axis = 1)

#merging the training set with the test set for preprocesing
data = train.append(test)
train_length = len(train)

#we will eb deleting variables which are no longer required to not eat up so much memory
del train
del test

In [0]:
#print('head', data.head())
#print('unique values', data.nunique())
#print('null values', data.isnull().sum())

In [26]:
#analyzing the column with too much of null values
z = pd.value_counts(data['Company-response'].iloc[:].values)
print(z.keys())

Index(['Company has responded to the consumer and the CFPB and chooses not to provide a public response',
       'Company chooses not to provide a public response',
       'Company believes it acted appropriately as authorized by contract or law',
       'Company believes the complaint is the result of a misunderstanding',
       'Company disputes the facts presented in the complaint',
       'Company believes complaint caused principally by actions of third party outside the control or direction of the company',
       'Company believes complaint is the result of an isolated error',
       'Company can't verify or dispute the facts in the complaint',
       'Company believes complaint represents an opportunity for improvement to better serve consumers',
       'Company believes complaint relates to a discontinued policy or procedure'],
      dtype='object')


In [0]:
#it seems that only top three have significant numbers so reducing the possible values of this column to 5 values i.e
# 0 if it has none of the top 3
# 1 if it has the first statement
# 2 if second
# 3 if third
# 4 if value was not known preiously

d1 = np.array((data['Company-response'] == z.keys()[0]).astype(np.int))
d2 = 2*np.array((data['Company-response'] == z.keys()[1]).astype(np.int))
d3 = 3*np.array((data['Company-response'] == z.keys()[2]).astype(np.int))
data['Company-response'] = d1 + d2 + d3

In [0]:
data['Company-response'] = data['Company-response'].fillna(4)
data['Consumer-disputes'] = data['Consumer-disputes'].fillna('unknown')# filling missing values

In [0]:
#the complainnt status certainly is not influenced by the dates they were sent but it may be influenced by the time it was delayed while being sent to the company

c = []
a1 = data['Date-sent-to-company'].iloc[:].values
b1 = data['Date-received'].iloc[:].values
for i in range(len(data)):
  a = str(a1[i]).split('/')
  b = str(b1[i]).split('/')
  c.append(int(a[2])*365 + int(a[1])*30 + int(a[0]) - int(b[2])*365 -int(b[1])*30 -int(b[0]))
    
    
data['delayed'] = c
data = data.drop(['Date-received', 'Date-sent-to-company'], axis = 1)
del a, a1
del b, b1
del c

In [0]:
# detecting the corresponding languages of summary

text = data['Consumer-complaint-summary'].iloc[:].values

"""!pip install langdetect
from langdetect import detect
languages = []
for i in range(len(text)):
  languages.append(detect(text[i]))
  
  
pd.DataFrame(languages, index = None).to_csv(os.path.join(path, 'languages.csv'), header = None, index = None)
"""
languages = pd.read_csv(os.path.join(path, 'languages.csv'), header = None)
languages.columns = ['one']
fr = {}
en = {}
es = {}
for i in range(len(languages.one)):
  if languages.one[i] == 'fr':
    fr.update({i:text[i]})
  elif languages.one[i] == 'es':
    es.update({i:text[i]})
  else:
    en.update({i:text[i]})
    
    


In [0]:
del d1
del d2 
del d3
del z

In [0]:
# langdetect detected more than three languages but we will consider these as main as their counts were substantially large, rest we shall append to english language
# now we make a dictionary of the statements corresponding to a particular language whilst preserving their indices

from nltk.corpus import stopwords
encor = {}
frcor = {}
escor = {}

swfr = set(stopwords.words('french'))
swen = set(stopwords.words('english'))
swes = set(stopwords.words('spanish'))


#making three different corpora for languages
for key in fr:
  words = fr[key].lower()
  words = words.split()
  ps = PorterStemmer()
  words = [ps.stem(word) for word in words if not word in swfr]
  sentence = ' '.join(words)
  frcor.update({key:sentence})

  
for key in en:
  words = en[key].lower()
  words = words.split()
  ps = PorterStemmer()
  words = [ps.stem(word) for word in words if not word in swen]
  sentence = ' '.join(words)
  encor.update({key:sentence})

  
for key in es:
  words = es[key].lower()
  words = words.split()
  ps = PorterStemmer()
  words = [ps.stem(word) for word in words if not word in swes]
  sentence = ' '.join(words)
  escor.update({key:sentence})


In [0]:
del en
del fr
del es

In [34]:
'''for key in encor:
  text[int(key)] = encor[key]
  
for key in frcor:
  text[int(key)] = frcor[key]
  
for key in escor:
  text[int(key)] = escor[key]'''

'for key in encor:\n  text[int(key)] = encor[key]\n  \nfor key in frcor:\n  text[int(key)] = frcor[key]\n  \nfor key in escor:\n  text[int(key)] = escor[key]'

In [0]:
# since counts of english are larger hence they get more words in their bag of words model
from sklearn.feature_extraction.text import CountVectorizer
encv = CountVectorizer(max_features = 4000)# english
frcv = CountVectorizer(max_features = 3000)#french
escv = CountVectorizer(max_features = 3000)#spanish

In [0]:
# we also append the language column to the main dataset so our model will know which language its currently dealing with

data['languages'] = languages 
data = data.drop(['Consumer-complaint-summary'], axis = 1)

In [0]:
# encoding the categorical features into numerical values

le = LabelEncoder()
for column in data.columns:
  data[column] = data[column].astype(str)
data = data.apply(le.fit_transform)
del languages

In [0]:
#creating our bow model
x1 = encv.fit_transform(encor.values()).toarray()
x2 = frcv.fit_transform(frcor.values()).toarray()
x3 = escv.fit_transform(escor.values()).toarray()

In [0]:
#this is done to maintain homogenity across our matrix and separate out these models as three different ones so they dont overlap
x1 = np.concatenate((x1, np.zeros((x1.shape[0], 6000))), axis = 1)
x2 = np.concatenate((np.zeros((x2.shape[0], 4000)), x2, np.zeros((x2.shape[0], 3000))), axis=1)
x3 = np.concatenate((np.zeros((x3.shape[0], 7000)), x3), axis = 1)


In [0]:
# while fitting count vectorizer they were converted to numpy arrays, now we have to return those to their original indices
X = np.empty((len(x1)+ len(x2) + len(x3), 10000))

In [0]:
#extracting keys of our corpus

encor = encor.keys()
frcor = frcor.keys()
escor = escor.keys()

In [0]:
# the following block of code returns the statements to their oringinal indices
i = 0
for key in encor:
  X[int(key)] = x1[i]
  i+=1

i = 0
for key in frcor:
  X[int(key)] = x2[i]
  i+=1
  
i = 0
for key in escor:
  X[int(key)] = x3[i]
  i+=1
  
np.savetxt(os.path.join(path, 'bow.txt'), X)

In [0]:
#concatenating the bag of words model and normal features
del x1, x2, x3
data = np.concatenate((data, X), axis = 1)

In [0]:
del X
np.savetxt(os.path.join(path, 'features.txt'), data)

In [64]:
#no matter how many variables i delete colab runs out of memory while training so we download features and will train them in a different session
from google.colab import files
files.download(os.path.join(path, 'features.txt'))

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 48582, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

m